In [226]:
from fbm import fbm, fgn, times
import numpy as np
from ipynb.fs.defs.Eke_Fractool_Python import FracTool


def MSE(N1,N2,N3):
    '''This function generates a matrix of known Hurst values and calculates the MSE of the expected Hurst and the 
    actual Hurst outputted by FracTool. The leftmost third of the matrix are fGn signals, the middle third are fBm 
    signals, and the last third are random arrays. 
    N1 = number of rows in matrix
    N2 = number of columns in matrix
    N3 = number of time-points in signal (3rd dimension in matrix)
    '''
    
    #first generate an matrix with known Hurst values and signal class
    
    #create arrays of expected Hurst and class (if interested in class as well)
    Hurst_expected = np.zeros((N1,N2))
    #Sig_class_expected = np.zeros((N1,N2))
    Signal_matrix = np.zeros((N1,N2,N3))
    fgn_section = np.floor(N2/3) #end of leftmost third of columns of matrix
    fbm_section = fgn_section + fgn_section #end of middle third of columns of matrix
   
    for i in range(N1):
        #leftmost third of matrix (fGn signals)
        for j in range(0,int(fgn_section)):
            Hurst_fgn = random.random() #generates random number between 0 and 1 (random Hurst value)
            Hurst_expected[i][j] = Hurst_fgn
            #Sig_class_expected[i][j] = 0 #fGn class
            #generate fGn signal with random Hurst at each element in first third of matrix
            Signal_matrix[i][j] = fgn(n=N3, hurst = Hurst_fgn, length=1, method='daviesharte')
        
        for k in range(int(fgn_section),int(fbm_section)):
            Hurst_fbm = random.random()
            Hurst_expected[i][k] = Hurst_fbm
            #Sig_class_expected[i][k] = 2 #fBm class
            #generate fBm signal with random Hurst at each element in middle third of matrix
            Signal_matrix[i][k] = fbm(n=N3+1, hurst = Hurst_fbm, length=1, method='daviesharte')[1:N3+1]

        for l in range(int(fbm_section), N2):
            Hurst_rand = 0.5 #expected Hurst for random signal?
            Hurst_expected[i][l] = Hurst_rand
            #Sig_class_expected[i][l] = 3 #outside model? 
            #generate random array (values between 0 and 1) of length N3 at each element in last third of matrix
            Signal_matrix[i][l] = list(np.random.rand(N3))
    
    #Now find the Hurst value and signal class that FracTool calculates on these known signals
    Hurst_actual = np.zeros((N1,N2))
    #Sig_class_actual = np.zeros((N1,N2))
    for n in range(N1): 
        for m in range(N2):
            result = FracTool(Signal_matrix[n,m])
            #result[0] of fractool is sig class
            #result[1] of fractool is Hurst coeff
            Hurst_actual[n,m] = result[1] 
            #Sig_class_actual[n,m] = result[0] #comment out if only interested in Hurst MSE
    #print(Hurst_expected)
    #print(Hurst_actual)
    #print(Sig_class_expected)
    #print(Sig_class_actual)
    
    #now calculate MSE between expected Hurst and actual Hurst calculated by FracTool
    summation = 0
    for x in range(N1):
        for y in range(N2): 
            diff = Hurst_actual[x,y] - Hurst_expected[x,y]
            sq_diff = np.nan_to_num(diff**2) #converts all nan values to 0 - removes nan from calculation
            summation = summation + sq_diff
    #MSE = summation/(N1*N2)  #need to divide by number that is included - omit all nan first? 
    MSE = summation/np.count_nonzero(np.nan_to_num(Hurst_actual))
    return MSE

#function call 
#ex:  N1=33, N2=33, N3 = 1024 generates 999 (33x33) signals of length = 1024 time-points
MSE(33,33,1024)

864


0.11404418792186882